<a href="https://colab.research.google.com/github/nateraw/download-musiccaps-dataset/blob/main/download_musiccaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Clips from the MusicCaps Dataset

In this notebook, we see how you can use `yt-dlp` to download clips from the [MusicCaps](https://huggingface.co/datasets/google/MusicCaps) dataset from Google. The MusicCaps dataset contains music and their associated text captions. You could use a dataset like this to train a text-to-audio generation model 😉. 

Once we've downloaded the clips, we'll explore them using a [Gradio](https://gradio.app/) interface.

If you like this notebook:

  - consider giving the [repo](https://github.com/nateraw/download-musiccaps-dataset) a star ⭐️
  - consider following me on Github [@nateraw](https://github.com/nateraw)

In [ ]:
%%capture
! pip install datasets[audio] yt-dlp

# For the interactive interface we'll need gradio
! pip install gradio

In [21]:
import subprocess
import os
from pathlib import Path

from datasets import load_dataset, Audio


def download_clip(
    video_identifier,
    output_filename,
    start_time,
    end_time,
    tmp_dir='./data/musiccaps',
    num_attempts=5,
    url_base='https://www.youtube.com/watch?v='
):
    import subprocess
    status = False

    command = f"""
        yt-dlp --quiet --force-keyframes-at-cuts --no-warnings -x --audio-format wav -f bestaudio -o "{output_filename}" --download-sections "*{start_time}-{end_time}" "{url_base}{video_identifier}" --cookies-from-browser chrome
    """.strip()

    attempts = 0
    while True:
        try:
            output = subprocess.check_output(command, shell=True,
                                                stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            raise err
            print("Error!", err.output)
            attempts += 1
            if attempts == num_attempts:
                return status, err.output
        else:
            break

    # Check if the video was successfully saved.
    status = os.path.exists(output_filename)
    return status, 'Downloaded'


def main(
    data_dir: str,
    sampling_rate: int = 44100,
    limit: int = None,
    num_proc: int = 1,
    writer_batch_size: int = 1,
    download_clip = download_clip
):
    """
    Download the clips within the MusicCaps dataset from YouTube.
    Args:
        data_dir: Directory to save the clips to.
        sampling_rate: Sampling rate of the audio clips.
        limit: Limit the number of examples to download.
        num_proc: Number of processes to use for downloading.
        writer_batch_size: Batch size for writing the dataset. This is per process.
    """

    ds = load_dataset('google/MusicCaps', split='train')
    if limit is not None:
        print(f"Limiting to {limit} examples")
        ds = ds.select(range(limit))

    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok=True, parents=True)

    def process(example):
        import os
        import subprocess
        outfile_path = str(data_dir / f"{example['ytid']}.wav")
        status = True
        if not os.path.exists(outfile_path):
            status = False
            status, log = download_clip(
                example['ytid'],
                outfile_path,
                example['start_s'],
                example['end_s'],
            )

        example['audio'] = outfile_path
        example['download_status'] = status
        return example

    return ds.map(
        process,
        num_proc=num_proc,
        writer_batch_size=writer_batch_size,
        keep_in_memory=False
    ).cast_column('audio', Audio(sampling_rate=sampling_rate))

## Load the Dataset

Here we are limiting to the first 32 examples. Since Colab is constrained to 2 cores, downloading the whole dataset here would take hours.

When running this on your own machine:
  - you can set `limit=None` to download + process the full dataset. Feel free to do that here in Colab, it'll just take a long time.
  - you should increase the `num_proc`, which will speed things up substantially
  - If you run out of memory, try reducing the `writer_batch_size`, as by default, it will keep 1000 examples in memory *per worker*.

In [22]:
ds = main('./music_data', num_proc=8, limit=32)

Limiting to 32 examples


Map (num_proc=8):   0%|          | 0/32 [00:04<?, ? examples/s]


CalledProcessError: Command 'yt-dlp --quiet --force-keyframes-at-cuts --no-warnings -x --audio-format wav -f bestaudio -o "music_data\-0Gj8-vB1q4.wav" --download-sections "*30-40" "https://www.youtube.com/watch?v=-0Gj8-vB1q4" --cookies-from-browser chrome' returned non-zero exit status 1.

In [14]:
ds.to_pandas().head()

,ytid,start_s,end_s,audioset_positive_labels,aspect_list,caption,author_id,is_balanced_subset,is_audioset_eval,audio,download_status
0,-0Gj8-vB1q4,30,40,"/m/0140xf,/m/02cjck,/m/04rlf","['low quality', 'sustained strings melody', 's...",The low quality recording features a ballad so...,4,False,True,"{'bytes': None, 'path': 'music_data\-0Gj8-vB1q...",False
1,-0SdAVK79lg,30,40,"/m/0155w,/m/01lyv,/m/0342h,/m/042v_gx,/m/04rlf...","['guitar song', 'piano backing', 'simple percu...",This song features an electric guitar as the m...,0,False,False,"{'bytes': None, 'path': 'music_data\-0SdAVK79l...",False
2,-0vPFx-wRRI,30,40,"/m/025_jnm,/m/04rlf","['amateur recording', 'finger snipping', 'male...",a male voice is singing a melody with changing...,6,False,True,"{'bytes': None, 'path': 'music_data\-0vPFx-wRR...",False
3,-0xzrMun0Rs,30,40,"/m/01g90h,/m/04rlf","['backing track', 'jazzy', 'digital drums', 'p...",This song contains digital drums playing a sim...,6,False,True,"{'bytes': None, 'path': 'music_data\-0xzrMun0R...",False
4,-1LrH01Ei1w,30,40,"/m/02p0sh1,/m/04rlf","['rubab instrument', 'repetitive melody on dif...",This song features a rubber instrument being p...,0,False,False,"{'bytes': None, 'path': 'music_data\-1LrH01Ei1...",False


Let's explore the samples using a quick Gradio Interface 🤗

In [5]:
import gradio as gr

def get_example(idx):
    ex = ds[idx]
    return ex['audio']['path'], ex['caption']

gr.Interface(
    get_example,
    inputs=gr.Slider(0, len(ds) - 1, value=0, step=1),
    outputs=['audio', 'textarea'],
    live=True
).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "d:\Anaconda\envs\NLP2\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\envs\NLP2\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\envs\NLP2\Lib\site-packages\gradio\blocks.py", line 2181, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\envs\NLP2\Lib\site-packages\gradio\blocks.py", line 1692, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\envs\NLP2\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^